In [6]:
print("Okay")

Okay


In [7]:
%pwd

'/home/yewinnaing/Ai-Training/research'

In [8]:
import os 
os.chdir('../')
%pwd


'/home/yewinnaing/Ai-Training'

In [11]:
from langchain.document_loaders import PyPDFLoader , DirectoryLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
def load_pdf_loader (data) : 
    loader = DirectoryLoader(data , glob="*.pdf" , loader_cls=PyPDFLoader )
    documents = loader.load() 
    return documents 

In [9]:
extractedData = load_pdf_loader(data="Data/")

In [ ]:
#extractedData


In [ ]:
def text_splite (extractedData) :
    text_splitter=RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap=20)
    text_chunk = text_splitter.split_documents(extractedData)
    return text_chunk 

In [11]:
txt_chunks = text_splite(extractedData)
print('Lenght of text chunks' , len(txt_chunks))

Lenght of text chunks 6356


In [6]:
from langchain_community.embeddings import HuggingFaceEmbeddings

def downloadHuggingFaceEmbedding():
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embedding

In [7]:
embeddings = downloadHuggingFaceEmbedding()

/tmp/ipykernel_17480/2029700098.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/yewinnaing/miniconda3/envs/my_chatbot_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
query_result = embeddings.embed_query("hello world")
print("lenght" , len(query_result))

lenght 384


In [2]:
from dotenv import load_dotenv 
load_dotenv()

True

In [ ]:
PINECONE_API_KEY= os.environ.get('PINECONE_API')


pcsk_77r8Kg_GipbZFJXfyyr72tgTHpuJXh1d5ZDuxuShjo9Dp13eGL5sqZJDS1QXrpmh2d219c


In [3]:
index_name = "chatbot"

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import Pinecone, ServerlessSpec
import os 

pc = Pinecone(api_key=PINECONE_API_KEY)




pc.create_index(
    name=index_name,
    dimension= 384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)


{
    "name": "chatbot",
    "metric": "cosine",
    "host": "chatbot-gj5tbq7.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [23]:
import os 
os.environ["PINECONE_API_KEY"] = "pcsk_77r8Kg_GipbZFJXfyyr72tgTHpuJXh1d5ZDuxuShjo9Dp13eGL5sqZJDS1QXrpmh2d219c"
os.environ["GROQ_API_KEY"] = "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"

In [18]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents= txt_chunks,
    index_name = index_name,
    embedding= embeddings
)

In [13]:
from langchain_pinecone import PineconeVectorStore

docsearch  = PineconeVectorStore.from_existing_index(
    index_name=index_name ,
    embedding=embeddings
)


In [35]:
retriver = docsearch.as_retriever(search_type="similarity" , search_kwargs={'k' : 3}) 

In [15]:
retrived_docs = retriver.invoke('What is Acne?')

In [16]:
retrived_docs

[Document(id='4cf98d3d-194c-46f9-9708-af836e1224cd', metadata={'author': "Ian B. Wilkinson,Tim Raine,Kate Wiles,Anna Goodhart,Catriona Hall,Harriet O'Neill", 'creationdate': '2017-05-02T19:09:54+01:00', 'creator': 'Adobe InDesign CS6 (Macintosh)', 'ebx_publisher': 'Oxford University Press USA', 'gts_pdfxconformance': 'PDF/X-1a:2001', 'gts_pdfxversion': 'PDF/X-1:2001', 'moddate': '2017-09-10T00:55:19+06:30', 'page': 216.0, 'page_label': '217', 'producer': 'Acrobat Distiller 10.1.16 (Macintosh)', 'source': 'Data/train_ai.pdf', 'title': 'Oxford Handbook of Clinical Medicine', 'total_pages': 911.0, 'trapped': '/False'}, page_content='pads; striae.\nAddison’s disease: Hyperpigmentation (face, neck, palmar creases).\nAcromegaly: Acral (distal) + soft tissue overgrowth; big jaws (macro-\ngnathia), hands and feet; the skin is thick; facial features are coarse.\nHyperandrogenism ( \ue033): Hirsutism; temporal balding; acne.\nHypopituitarism:  Pale or yellow tinged thinned skin, resulting in \nﬁ

In [13]:
# from langchain_openai import OpenAI
# llm = OpenAI(temperature=0.4 , max_tokens=500) 

In [ ]:
# from langchain.chains import create_retrieval_chain
# from langchain.chains.combine_documents import  create_stuff_documents_chain 
# from langchain.prompts import ChatPromptTemplate

# systemPrompt = (
#     "You are an assitant for the question-answering task"
#     "Use the fllowing piceces of reterived context to answer"
#     "the question. If you don't know the answer , say that you "
#     "don't know . Use three sentense maximum and keep the "
#     "answer conices" 
#     "\n\n" 
#     "{context}"
# )

# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system" , systemPrompt)
#         ('human' , "{input}")
#     ]
# )
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [14]:
# questionAnswerChain = create_stuff_documents_chain(llm , prompt )
# ragChain = create_retrieval_chain(retriver , questionAnswerChain)

In [ ]:
response = ragChain.invoke({"input" : "What is Actne ? "})
print(response['answer'])

In [47]:
from langchain_groq import ChatGroq
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

# Initialize the LLM
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.0,
    max_retries=2,
)

# Define the system prompt
systemPrompt = (
    "You are an assistant for the question-answering task. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise. "
    "Always answer in the Myanmar (Burmese) language."
    "\n\n"
    "{context}"
)

# Create the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", systemPrompt),
        ("human", "{input}")
    ]
)

# Create the question-answering chain using the LLM and the prompt
questionAnswerChain = create_stuff_documents_chain(llm, prompt)

# retriver = docsearch.as_retriever(search_type="similarity" , search_kwargs={'k' : 3}) 
ragChain = create_retrieval_chain(retriver, questionAnswerChain)

# Use the chain to get the answer to a question
response = ragChain.invoke({"input": "What is Acne"})

# Print the response
print(response['answer'])


အက်စ်နီ (Acne) ဆိုသည်မှာ အရေပြားပေါ်တွင် ဖြစ်ပေါ်လာသော အနာအများများ ဖြစ်သည်။ အက်စ်နီ ဖြစ်ရန် အကြောင်းများစွာ ရှိသည်။ အက်စ်နီ ဖြစ်ရန် အကြောင်းတစ်ခုမှာ ဟိုင်းပါအန်ဒြိုဂျင် (Hyperandrogenism) ဖြစ်သည်။ ဟိုင်းပါအန်ဒြိုဂျင် ရှိသူများတွင် အက်စ်နီ အများအပြား ဖြစ်ပေါ်လေ့ ရှိသည်။


In [ ]:
import mysql.connector
# db = mysql.connector.connect(
#     host="127.0.0.1",
#     user="root",
#     password="yourpassword",
#     database="play_ground"
# )
cursor = db.cursor()
cursor.execute("SELECT * FROM users")
rows = cursor.fetchall()
# Convert rows to text format
extractedData = "\n".join([str(row[0]) for row in rows])


# Close database connection
cursor.close()
db.close()

In [15]:
# embeddings = downloadHuggingFaceEmbedding()
# txt_chunks = text_splite(extractedData)

In [ ]:
pc.create_index(
    name=index_name,
    dimension=384,  # Replace with your model dimensions
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)